### 라이브러리 불러오기

In [11]:
# 숫자 및 데이터 다루는 라이브러리
import numpy as np
import pandas as pd

# 분류모델 선언
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

# 훈련/테스트 데이터 자동 분리
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


# 분류모델 정확도 라이브러리 선언
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# 데이터 시각화
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import export_text
from sklearn.tree import plot_tree

### 데이터 불러오기

In [28]:
# csv 파일 불러오기_1
ydp1 = pd.read_csv("../dataset/건대_1.csv")
ydp1

,name,gubun,score,visitCount,blogCount
0,솥돈 영등포점,돼지고기구이,4.47/5,방문자 리뷰_ 232,블로그 리뷰_ 97
1,해성중국요리,중식당,4.36/5,방문자 리뷰_ 264,블로그 리뷰_ 348
2,진주집,국수,4.4/5,"방문자 리뷰_ 5,569","블로그 리뷰_ 3,332"
3,정인면옥 본점,냉면,4.24/5,"방문자 리뷰_ 1,777","블로그 리뷰_ 1,598"
4,세상의 모든아침,브런치,4.26/5,"방문자 리뷰_ 2,668","블로그 리뷰_ 3,862"
...,...,...,...,...,...
246,슈퍼말차 더현대서울,"카페,디저트",4.33/5,방문자 리뷰_ 248,블로그 리뷰_ 286
247,리춘시장 영등포역점,중식당,4.23/5,방문자 리뷰_ 553,블로그 리뷰_ 165
248,더스크블루,"카페,디저트",4.89/5,방문자 리뷰_ 160,블로그 리뷰_ 203
249,어펜딕스,브런치,4.44/5,방문자 리뷰_ 622,블로그 리뷰_ 107


### 데이터 전처리 작업

In [29]:
# 필요없는 문자 삭제
ydp1.score = ydp1.score.str[0:-2]
ydp1.visitCount = ydp1.visitCount.str[7:]
ydp1.blogCount = ydp1.blogCount.str[7:]

In [30]:
# 콤마 삭제 & int 캐스팅
ydp1.visitCount = ydp1.visitCount.str.replace(',', '').astype('int64')
ydp1.blogCount = ydp1.blogCount.str.replace(',', '').astype('int64')

In [31]:
ydp1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        251 non-null    object
 1   gubun       251 non-null    object
 2   score       251 non-null    object
 3   visitCount  251 non-null    int64 
 4   blogCount   251 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 9.9+ KB


In [32]:
# 숫자형 자료 캐스팅
ydp1["score"] = ydp1.score.astype(float)
ydp1["visitCount"] = ydp1.visitCount.astype(int)
ydp1["blogCount"] = ydp1.blogCount.astype(int)

In [33]:
# csv 파일 불러오기_2
ydp2 = pd.read_csv("../dataset/건대_2.csv")
ydp2

,name,tenCen,twoCen,threeCen,fourCen,fiveCen,sixCen,womanRatio,manRatio
0,코지로 선유도점,25.00000000000001%_10|,93.57799999999996%_20|,85.82199999999997%_30|,71.33399999999999%_40|,67.30099999999997%_50|,63.16449999999999%_60|,여자_45%,남자_55%
1,냅다청양집 본점,38.883999999999965%_10|,99.31399999999998%_20|,92.50799999999995%_30|,84.2995%_40|,70.39099999999996%_50|,67.228%_60|,여자_46%,남자_54%
2,진주집,24.87366666666668%_10|,90.54299999999998%_20|,95.34333333333332%_30|,87.98266666666665%_40|,85.11233333333334%_50|,76.34999999999998%_60|,여자_49%,남자_51%
3,정인면옥 본점,27.79500000000001%_10|,83.47633333333333%_20|,86.40133333333331%_30|,81.48466666666664%_40|,81.82333333333331%_50|,76.86899999999999%_60|,여자_38%,남자_62%
4,세상의 모든아침,27.79500000000001%_10|,83.47633333333333%_20|,86.40133333333331%_30|,81.48466666666664%_40|,81.82333333333331%_50|,76.86899999999999%_60|,여자_38%,남자_62%
...,...,...,...,...,...,...,...,...,...
154,폴트버거 더현대서울,35.50188235294117%_10|,50.68329411764706%_20|,43.68729411764706%_30|,35.826352941176474%_40|,23.713176470588234%_50|,20.835764705882344%_60|,여자_51%,남자_49%
155,미나리우리삼겹,25.00000000000001%_10|,96.87299999999995%_20|,94.41599999999997%_30|,83.44249999999994%_40|,69.18749999999999%_50|,63.904999999999966%_60|,여자_48%,남자_52%
156,더플레이스 영등포 타임스퀘어점,67.49399999999997%_10|,99.48449999999994%_20|,92.58499999999994%_30|,69.43999999999998%_40|,47.08899999999998%_50|,40.301499999999976%_60|,여자_51%,남자_49%
157,온더보더 여의도IFC점,18.41011764705881%_10|,45.065882352941166%_20|,48.81835294117648%_30|,42.14682352941175%_40|,27.282823529411775%_50|,22.92941176470589%_60|,여자_56%,남자_44%


In [34]:
# 필요없는 문자 삭제
ydp2.tenCen = ydp2.tenCen.str[0:-5]
ydp2.twoCen = ydp2.twoCen.str[0:-5]
ydp2.threeCen = ydp2.threeCen.str[0:-5]
ydp2.fourCen = ydp2.fourCen.str[0:-5]
ydp2.fiveCen = ydp2.fiveCen.str[0:-5]
ydp2.sixCen = ydp2.sixCen.str[0:-5]

In [35]:
# 필요없는 문자 삭제
ydp2.womanRatio = ydp2.womanRatio.str[0:-1]
ydp2.womanRatio = ydp2.womanRatio.str[-2:]

In [36]:
# 필요없는 문자 삭제
ydp2.manRatio = ydp2.manRatio.str[0:-1]
ydp2.manRatio = ydp2.manRatio.str[-2:]

In [37]:
ydp2

,name,tenCen,twoCen,threeCen,fourCen,fiveCen,sixCen,womanRatio,manRatio
0,코지로 선유도점,25.00000000000001,93.57799999999996,85.82199999999997,71.33399999999999,67.30099999999997,63.16449999999999,45,55
1,냅다청양집 본점,38.883999999999965,99.31399999999998,92.50799999999995,84.2995,70.39099999999996,67.228,46,54
2,진주집,24.87366666666668,90.54299999999998,95.34333333333332,87.98266666666665,85.11233333333334,76.34999999999998,49,51
3,정인면옥 본점,27.79500000000001,83.47633333333333,86.40133333333331,81.48466666666664,81.82333333333331,76.86899999999999,38,62
4,세상의 모든아침,27.79500000000001,83.47633333333333,86.40133333333331,81.48466666666664,81.82333333333331,76.86899999999999,38,62
...,...,...,...,...,...,...,...,...,...
154,폴트버거 더현대서울,35.50188235294117,50.68329411764706,43.68729411764706,35.826352941176474,23.713176470588234,20.835764705882344,51,49
155,미나리우리삼겹,25.00000000000001,96.87299999999995,94.41599999999997,83.44249999999994,69.18749999999999,63.904999999999966,48,52
156,더플레이스 영등포 타임스퀘어점,67.49399999999997,99.48449999999994,92.58499999999994,69.43999999999998,47.08899999999998,40.301499999999976,51,49
157,온더보더 여의도IFC점,18.41011764705881,45.065882352941166,48.81835294117648,42.14682352941175,27.282823529411775,22.92941176470589,56,44


In [38]:
ydp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        159 non-null    object
 1   tenCen      159 non-null    object
 2   twoCen      159 non-null    object
 3   threeCen    159 non-null    object
 4   fourCen     159 non-null    object
 5   fiveCen     159 non-null    object
 6   sixCen      159 non-null    object
 7   womanRatio  159 non-null    object
 8   manRatio    159 non-null    object
dtypes: object(9)
memory usage: 11.3+ KB


In [39]:
ydp2["tenCen"] = ydp2.tenCen.astype(float)
ydp2["twoCen"] = ydp2.twoCen.astype(float)
ydp2["threeCen"] = ydp2.threeCen.astype(float)
ydp2["fourCen"] = ydp2.fourCen.astype(float)
ydp2["fiveCen"] = ydp2.fiveCen.astype(float)
ydp2["sixCen"] = ydp2.sixCen.astype(float)

ydp2["womanRatio"] = ydp2.womanRatio.astype(int)
ydp2["manRatio"] = ydp2.manRatio.astype(int)

In [40]:
ydp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        159 non-null    object 
 1   tenCen      159 non-null    float64
 2   twoCen      159 non-null    float64
 3   threeCen    159 non-null    float64
 4   fourCen     159 non-null    float64
 5   fiveCen     159 non-null    float64
 6   sixCen      159 non-null    float64
 7   womanRatio  159 non-null    int32  
 8   manRatio    159 non-null    int32  
dtypes: float64(6), int32(2), object(1)
memory usage: 10.1+ KB


In [41]:
ydp2["tenCen"] = ydp2.tenCen.round(2)
ydp2["twoCen"] = ydp2.twoCen.round(2)
ydp2["threeCen"] = ydp2.threeCen.round(2)
ydp2["fourCen"] = ydp2.fourCen.round(2)
ydp2["fiveCen"] = ydp2.fiveCen.round(2)
ydp2["sixCen"] = ydp2.sixCen.round(2)

In [42]:
ydp2

,name,tenCen,twoCen,threeCen,fourCen,fiveCen,sixCen,womanRatio,manRatio
0,코지로 선유도점,25.00,93.58,85.82,71.33,67.30,63.16,45,55
1,냅다청양집 본점,38.88,99.31,92.51,84.30,70.39,67.23,46,54
2,진주집,24.87,90.54,95.34,87.98,85.11,76.35,49,51
3,정인면옥 본점,27.80,83.48,86.40,81.48,81.82,76.87,38,62
4,세상의 모든아침,27.80,83.48,86.40,81.48,81.82,76.87,38,62
...,...,...,...,...,...,...,...,...,...
154,폴트버거 더현대서울,35.50,50.68,43.69,35.83,23.71,20.84,51,49
155,미나리우리삼겹,25.00,96.87,94.42,83.44,69.19,63.90,48,52
156,더플레이스 영등포 타임스퀘어점,67.49,99.48,92.58,69.44,47.09,40.30,51,49
157,온더보더 여의도IFC점,18.41,45.07,48.82,42.15,27.28,22.93,56,44


In [43]:
# 데이터 join
ydpJoin = pd.merge(left=ydp1, right=ydp2, on=["name"], how="inner")
ydpJoin

,name,gubun,score,visitCount,blogCount,tenCen,twoCen,threeCen,fourCen,fiveCen,sixCen,womanRatio,manRatio
0,진주집,국수,4.40,5569,3332,24.87,90.54,95.34,87.98,85.11,76.35,49,51
1,정인면옥 본점,냉면,4.24,1777,1598,27.80,83.48,86.40,81.48,81.82,76.87,38,62
2,세상의 모든아침,브런치,4.26,2668,3862,27.80,83.48,86.40,81.48,81.82,76.87,38,62
3,갈빗,"육류,고기요리",4.49,973,918,18.22,78.00,70.94,63.56,56.96,54.03,44,56
4,대한옥,"곰탕,설렁탕",4.36,1331,1613,18.22,78.00,70.94,63.56,56.96,54.03,44,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,폴트버거 더현대서울,햄버거,3.98,132,409,35.50,50.68,43.69,35.83,23.71,20.84,51,49
151,미나리우리삼겹,"육류,고기요리",4.70,672,358,25.00,96.87,94.42,83.44,69.19,63.90,48,52
152,더플레이스 영등포 타임스퀘어점,이탈리아음식,4.46,3620,607,67.49,99.48,92.58,69.44,47.09,40.30,51,49
153,온더보더 여의도IFC점,"멕시코,남미음식",4.40,2510,1086,18.41,45.07,48.82,42.15,27.28,22.93,56,44


### CountVectorizer 사용_카테고리 정리

In [33]:
cvect = CountVectorizer()
dtm = cvect.fit_transform(ydpJoin["gubun"])

In [34]:
# gubun에 들어있는 내용들 구분자, 띄어쓰기 기준으로 분류(중복도 제거해줌)
feature_names = cvect.get_feature_names()

C:\Users\SMART18\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [35]:
# for문에서 사용하기 위해서 list로 담아줌
feature_namesList = list(feature_names)

In [36]:
# 카테고리 분류
categoryDic = {"이자카야": "주류"
,"오므라이스": "아시아음식"
,"김밥": "한식"
,"피자":"양식"
,"포장마차": "주류"
,"패밀리레스토랑": "뷔페"
,"스테이크":"양식"
,"설렁탕": "한식"
,"블루보틀":"카페"
,"케이크": "카페"
,"국밥": "한식"
,"칼국수": "한식"
,"만두": "한식"               
,"퓨전음식": "기타"
,"카페": "카페"
,"디저트": "카페"
,"베이커리": "카페"
,"베트남음식": "아시아음식"
,"족발": "한식"
,"보쌈": "한식"
,"백숙": "한식"
,"백숙삼계탕": "한식"
,"찌개": "한식"
,"전골": "한식"
,"한식": "한식"
,"종합분식": "분식"
,"분식": "분식"
,"이탈리아음식": "양식"
,"브런치": "카페"
,"덮밥": "일식"
,"쭈꾸미요리": "한식" 
,"냉면": "한식"
,"장어": "한식"
,"먹장어": "한식"
,"양식": "양식"
,"멕시코": "양식"
,"남미음식": "양식"
,"중식": "중식"
,"중식당": "중식"
,"곰탕": "한식"
,"설렁탕": "한식"
,"치킨": "한식"
,"닭강정": "한식"
,"요리주점": "주류"
,"햄버거": "양식"
,"순대": "한식"
,"순대국": "한식"
,"전": "한식"
,"빈대떡": "한식"
,"국밥": "한식"
,"양꼬치": "중식"
,"태국음식": "아시아음식"
,"돼지고기구이": "한식"
,"초밥": "일식"
,"롤": "일식"
,"샤브샤브": "한식"
,"소고기구이": "한식"
,"만두": "한식"
,"돈가스": "분식"
,"곱창": "한식"
,"막창": "한식"
,"양": "한식"
,"스파게티": "양식"
,"파스타전문": "양식"
,"일식당": "일식"
,"우동" : "일식"
,"소바" : "일식"
,"칼국수": "한식"
,"만두" : "한식"
,"다이어트": "기타"
,"샐러드": "기타"
,"아시아음식": "아시아음식"
,"아귀찜": "한식"
,"해물찜": "한식"
,"감자탕": "한식"
,"추어탕": "한식"
,"일본식라면": "일식"
,"생선구이": "한식"
,"양갈비": "기타"
,"국밥": "한식"
,"두부요리": "한식"
,"찌개": "한식"
,"전골": "한식"
,"돼지고기구이": "한식"
,"햄버거": "양식"
,"멕시코": "양식"
,"남미음식": "양식"
,"정육식당": "한식" 
,"육류": "한식"
,"고기요리" : "한식"
,"맥주": "주류"
,"호프": "주류"
,"국수": "한식"
,"닭갈비": "한식"
,"생선회": "일식"
,"일식튀김": "일식"
,"닭발": "한식"
,"술집": "주류"
,"닭볶음탕": "한식"
,"떡볶이": "분식"
,"카페": "카페"
,"대게요리": "기타"
,"피자": "양식"
,"양꼬치": "중식"
,"백반": "한식"
,"가정식": "한식"
,"전": "한식"
,"빈대떡" : "한식"
,"닭발": "한식"
,"게요리": "기타"
,"양식": "양식"
,"베이글": "카페"
,"브런치": "카페"
,"도넛": "카페"
,"케이크": "카페"
,"베이커리": "카페"
,"케이크전문": "카페"
,"스페인음식": "양식"
,"BAR": "주류"
,"바": "주류"
,"bar": "주류"
,"딤섬": "중식"
,"중식만두": "중식"
,"일식튀김": "일식"
,"꼬치": "일식"
,"프랑스음식": "양식",
"인도음식": "아시아음식",
"터키음식": "아시아음식",
"와인": "주류",
"아이스크림"  :"카페" ,
"카레" : "아시아음식",
"카페": "카페"
,"디저트" : "카페"
,"차": "카페"
,"뷔페": "뷔페"
,"주꾸미요리": "한식"
,"이북음식": "한식"
,"프랑스음식": "양식"
,"홍차전문점": "카페"
,"해물": "한식"
,"샌드위치": "카페"
,"빙수": "카페"
,"기사식당": "한식"
,"뷔페": "뷔페"
,"매운탕": "한식"
,"스페인음식": "양식"
,"한정식":"한식"
,"바\(BAR\)": "주류"
,"바닷가재요리": "양식"
,"백숙삼계탕": "한식"
,"페밀리레스토랑": "뷔페"
,"아귀찜": "한식"
,"추어탕": "한식"
,"생선구이": "한식"
,"두부요리": "한식"
,"정육식당": "한식"
,"술집": "주류"
,"닭볶음탕": "한식"
,"대게요리": "한식"
,"게요리": "한식"
,"민속주점" : "주류"}

In [37]:
# for문에서 사용하기 위해서 list로 담아줌(key값 : gubun)
categoryDicList = list(categoryDic)

In [38]:
# for문에서 사용하기 위해서 list로 담아줌(value값 : category)
categoryDicReverse = list(categoryDic.values())

In [39]:
# 카테고리 생성
for j in range (0, len(feature_namesList)):
    for i in range (0, len(categoryDicList)):
        if (feature_namesList[j] == categoryDicList[i]):
            key = "string"
            keyReplace = key.replace("string",feature_namesList[j])
            value = categoryDic[keyReplace]
            print(value)
            ydpJoin.loc[ydpJoin["gubun"].str.contains(categoryDicList[i]), "category"] = value

주류
한식
한식
한식
한식
한식
양식
한식
기타
한식
한식
카페
분식
한식
한식
주류
양식
양식
한식
카페
카페
한식
뷔페
카페
카페
카페
기타
한식
한식
일식
한식
양식
양식
기타
양식
주류
주류
일식
한식
주류
양식
아시아음식
일식
일식
한식
분식
한식
중식
일식
카페
한식
카페
아시아음식
양식
뷔페
주류
양식
한식
양식
주류


In [40]:
# 카테고리 분류 안된 Nan값 찾기 위해서 변수 생성
findCategory = ydpJoin.category

In [41]:
# Nan값 찾기
findNan = ydpJoin.loc[((findCategory != "한식") & (findCategory != "양식") & (findCategory != "아시아음식") & (findCategory != "일식") & (findCategory != "중식") & (findCategory != "분식") & (findCategory != "카페") & (findCategory != "뷔페") & (findCategory != "주류") & (findCategory != "기타"))]

In [42]:
# Nan값 변경위해 gubun 리스트 생성
findNanList = list(findNan.gubun)

In [43]:
# Nan값 기타로 변경
for i in range (0, len(findNanList)):
        findNan.loc[findNan["gubun"].str.contains(findNanList[i]), "category"] = "기타"

C:\Users\SMART18\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\SMART18\AppData\Local\Temp/ipykernel_8224/3533570633.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  findNan.loc[findNan["gubun"].str.contains(findNanList[i]), "category"] = "기타"


In [44]:
# 기존 데이터프레임에서 Nan값이 아닌 카테고리 찾기
findNotNan = ydpJoin.loc[~((findCategory != "한식") & (findCategory != "양식") & (findCategory != "아시아음식") & (findCategory != "일식") & (findCategory != "중식") & (findCategory != "분식") & (findCategory != "카페") & (findCategory != "뷔페") & (findCategory != "주류") & (findCategory != "기타"))]

In [45]:
# 데이터 합치기(기존, 기타)
joinCatagory = pd.concat([findNan, findNotNan], axis=0, ignore_index=True)

In [46]:
joinCatagory

,name,gubun,score,visitCount,blogCount,name2,tenCen,twoCen,threeCen,fourCen,fiveCen,sixCen,womanRatio,manRatio,category
0,골목집,오리요리,4.48,1533,662,골목집,25.00,95.69,95.64,92.16,91.82,97.92,43,57,기타
1,콘래드서울 버티고,바(BAR),4.29,978,767,콘래드서울 버티고,26.30,78.31,78.72,66.63,54.57,41.43,56,44,NaN
2,여로집 영등포점,오징어요리,4.33,1222,803,여로집 영등포점,22.36,46.65,47.22,42.39,30.83,26.33,54,46,기타
3,계옥정,닭요리,4.33,848,163,계옥정,25.00,90.96,88.65,84.31,73.27,68.97,49,51,기타
4,호텔707,바(BAR),4.41,802,1642,호텔707,24.33,50.69,42.56,32.60,22.74,16.05,55,45,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,폴트버거 더현대서울,햄버거,3.98,132,409,폴트버거 더현대서울,35.50,50.68,43.69,35.83,23.71,20.84,51,49,양식
151,미나리우리삼겹,"육류,고기요리",4.70,672,358,미나리우리삼겹,25.00,96.87,94.42,83.44,69.19,63.90,48,52,한식
152,더플레이스 영등포 타임스퀘어점,이탈리아음식,4.46,3620,607,더플레이스 영등포 타임스퀘어점,67.49,99.48,92.58,69.44,47.09,40.30,51,49,양식
153,온더보더 여의도IFC점,"멕시코,남미음식",4.40,2510,1086,온더보더 여의도IFC점,18.41,45.07,48.82,42.15,27.28,22.93,56,44,양식


In [47]:
# 바(BAR)는 괄호때문에 인식을 못해서 따로 변경
joinCatagory.loc[joinCatagory.gubun.str.contains("바\(BAR\)"),"category"] = "주류"

In [48]:
joinCatagory

,name,gubun,score,visitCount,blogCount,name2,tenCen,twoCen,threeCen,fourCen,fiveCen,sixCen,womanRatio,manRatio,category
0,골목집,오리요리,4.48,1533,662,골목집,25.00,95.69,95.64,92.16,91.82,97.92,43,57,기타
1,콘래드서울 버티고,바(BAR),4.29,978,767,콘래드서울 버티고,26.30,78.31,78.72,66.63,54.57,41.43,56,44,주류
2,여로집 영등포점,오징어요리,4.33,1222,803,여로집 영등포점,22.36,46.65,47.22,42.39,30.83,26.33,54,46,기타
3,계옥정,닭요리,4.33,848,163,계옥정,25.00,90.96,88.65,84.31,73.27,68.97,49,51,기타
4,호텔707,바(BAR),4.41,802,1642,호텔707,24.33,50.69,42.56,32.60,22.74,16.05,55,45,주류
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,폴트버거 더현대서울,햄버거,3.98,132,409,폴트버거 더현대서울,35.50,50.68,43.69,35.83,23.71,20.84,51,49,양식
151,미나리우리삼겹,"육류,고기요리",4.70,672,358,미나리우리삼겹,25.00,96.87,94.42,83.44,69.19,63.90,48,52,한식
152,더플레이스 영등포 타임스퀘어점,이탈리아음식,4.46,3620,607,더플레이스 영등포 타임스퀘어점,67.49,99.48,92.58,69.44,47.09,40.30,51,49,양식
153,온더보더 여의도IFC점,"멕시코,남미음식",4.40,2510,1086,온더보더 여의도IFC점,18.41,45.07,48.82,42.15,27.28,22.93,56,44,양식


### 라벨링 작업

In [49]:
joinCatagory.category.drop_duplicates()

0        기타
1        주류
5        한식
7        카페
10       양식
11       일식
23       중식
29    아시아음식
42       분식
49       뷔페
Name: category, dtype: object

In [50]:
joinCatagory["category"].value_counts()

한식       46
카페       28
양식       23
일식       21
주류       16
중식        9
기타        5
뷔페        3
아시아음식     2
분식        2
Name: category, dtype: int64

In [51]:
labelDict = joinCatagory.category.drop_duplicates().\
             reset_index(drop=True).to_dict()

In [52]:
labelDict

{0: '기타',
 1: '주류',
 2: '한식',
 3: '카페',
 4: '양식',
 5: '일식',
 6: '중식',
 7: '아시아음식',
 8: '분식',
 9: '뷔페'}

In [53]:
# 많은 순으로 재라벨링
labelDict = \
{0: '분식',
 1: '아시아음식',
 2: '뷔페',
 3: '기타',
 4: '중식',
 5: '주류',
 6: '일식',
 7: '양식',
 8: '카페',
 9: '한식'}

In [54]:
# reverse
labelDict = dict(map(reversed,labelDict.items()))

In [55]:
labelDict

{'분식': 0,
 '아시아음식': 1,
 '뷔페': 2,
 '기타': 3,
 '중식': 4,
 '주류': 5,
 '일식': 6,
 '양식': 7,
 '카페': 8,
 '한식': 9}

In [56]:
joinCatagory.category.map(labelDict)

0      3
1      5
2      3
3      3
4      5
      ..
150    7
151    9
152    7
153    7
154    5
Name: category, Length: 155, dtype: int64

In [57]:
joinCatagory["labelcode"] = joinCatagory.category.map(labelDict)

In [58]:
joinCatagory

,name,gubun,score,visitCount,blogCount,name2,tenCen,twoCen,threeCen,fourCen,fiveCen,sixCen,womanRatio,manRatio,category,labelcode
0,골목집,오리요리,4.48,1533,662,골목집,25.00,95.69,95.64,92.16,91.82,97.92,43,57,기타,3
1,콘래드서울 버티고,바(BAR),4.29,978,767,콘래드서울 버티고,26.30,78.31,78.72,66.63,54.57,41.43,56,44,주류,5
2,여로집 영등포점,오징어요리,4.33,1222,803,여로집 영등포점,22.36,46.65,47.22,42.39,30.83,26.33,54,46,기타,3
3,계옥정,닭요리,4.33,848,163,계옥정,25.00,90.96,88.65,84.31,73.27,68.97,49,51,기타,3
4,호텔707,바(BAR),4.41,802,1642,호텔707,24.33,50.69,42.56,32.60,22.74,16.05,55,45,주류,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,폴트버거 더현대서울,햄버거,3.98,132,409,폴트버거 더현대서울,35.50,50.68,43.69,35.83,23.71,20.84,51,49,양식,7
151,미나리우리삼겹,"육류,고기요리",4.70,672,358,미나리우리삼겹,25.00,96.87,94.42,83.44,69.19,63.90,48,52,한식,9
152,더플레이스 영등포 타임스퀘어점,이탈리아음식,4.46,3620,607,더플레이스 영등포 타임스퀘어점,67.49,99.48,92.58,69.44,47.09,40.30,51,49,양식,7
153,온더보더 여의도IFC점,"멕시코,남미음식",4.40,2510,1086,온더보더 여의도IFC점,18.41,45.07,48.82,42.15,27.28,22.93,56,44,양식,7


In [44]:
# ----------------------------------------------------------------------------------------

In [199]:
corrDf = joinCatagory.corr()

In [200]:
corrDf

,score,visitCount,blogCount,tenCen,twoCen,threeCen,fourCen,fiveCen,sixCen,womanRatio,manRatio,labelcode,sex
score,1.000000,-0.059646,-0.040310,-0.058526,0.001149,-0.074173,-0.166445,-0.170578,-0.194700,0.072254,-0.072254,0.148089,-0.038033
visitCount,-0.059646,1.000000,0.476548,0.114923,-0.077332,-0.091604,-0.069874,-0.066152,-0.053896,0.121667,-0.121667,0.012447,0.118793
blogCount,-0.040310,0.476548,1.000000,-0.059295,-0.021894,-0.015229,0.006182,0.072444,0.066868,-0.089834,0.089834,0.095997,-0.071150
tenCen,-0.058526,0.114923,-0.059295,1.000000,0.576854,0.435273,0.368274,0.260825,0.178147,0.235205,-0.235205,0.025604,0.179039
twoCen,0.001149,-0.077332,-0.021894,0.576854,1.000000,0.908108,0.771678,0.655488,0.530490,0.012271,-0.012271,0.052589,-0.027671
threeCen,-0.074173,-0.091604,-0.015229,0.435273,0.908108,1.000000,0.950134,0.865621,0.749190,-0.199629,0.199629,0.137042,-0.225756
fourCen,-0.166445,-0.069874,0.006182,0.368274,0.771678,0.950134,1.000000,0.959722,0.857475,-0.302742,0.302742,0.151668,-0.327788
fiveCen,-0.170578,-0.066152,0.072444,0.260825,0.655488,0.865621,0.959722,1.000000,0.926691,-0.407256,0.407256,0.160298,-0.408971
sixCen,-0.194700,-0.053896,0.066868,0.178147,0.530490,0.749190,0.857475,0.926691,1.000000,-0.464871,0.464871,0.116047,-0.444265
womanRatio,0.072254,0.121667,-0.089834,0.235205,0.012271,-0.199629,-0.302742,-0.407256,-0.464871,1.000000,-1.000000,-0.056018,0.801060


In [223]:
stdCorr = 0.4

In [224]:
corrDf.loc[abs(corrDf.labelcode > stdCorr) & (corrDf.labelcode != 1)].index

Index([], dtype='object')

In [205]:
features = list(corrDf.loc[(corrDf.labelcode > stdCorr) & (corrDf.labelcode != 1)].index)

In [206]:
features

['score', 'threeCen', 'fourCen', 'fiveCen', 'sixCen']

In [207]:
joinCatagory.shape

(155, 17)

In [208]:
train_test_split(joinCatagory, test_size=0.2)

[            name   gubun  score  visitCount  blogCount        name2  \
 120         본보야지  이탈리아음식   4.40         386        969         본보야지   
 135          스시젠     일식당   4.78         969        192          스시젠   
 39           채윤희    포장마차   4.32         416        536          채윤희   
 65        너도나도식당      한식   4.34         761        371       너도나도식당   
 86          귀비주옥     중식당   4.41         170        105         귀비주옥   
 ..           ...     ...    ...         ...        ...          ...   
 128           준무    이자카야   4.31         253        286           준무   
 22      러스트 베이커리    베이커리   4.42        4059       2153     러스트 베이커리   
 104      차알 여의도점     중식당   4.35        1182        600      차알 여의도점   
 44   잠수교집 직영 문래점  돼지고기구이   4.53         754        447  잠수교집 직영 문래점   
 75          이한스시    초밥,롤   4.38        1669       1457         이한스시   
 
            tenCen        twoCen      threeCen       fourCen       fiveCen  \
 120  1.500083e+08  3.281510e+08  3.088689e+08  2.967304

In [209]:
trainingData_features,\
testData_features,\
trainingData_label,\
testData_label =\
train_test_split(joinCatagory.loc[:,features],
                 joinCatagory.loc[:,label],
                 test_size=0.2,
                 random_state=5)

In [210]:
print(trainingData_features.shape)
print(testData_features.shape)
print(trainingData_label.shape)
print(testData_label.shape)

(124, 5)
(31, 5)
(124, 1)
(31, 1)


In [211]:
model_method = svm.SVC(random_state=1, C=100, kernel="rbf", gamma=10)

In [212]:
model = model_method.fit(trainingData_features, trainingData_label)

C:\Users\SMART18\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [213]:
predictValue = model.predict(testData_features)

In [214]:
len(predictValue)

31

In [215]:
model.score(testData_features, testData_label)

0.1935483870967742

In [216]:
testIndex = testData_label.index

In [217]:
testDataAll = joinCatagory.loc[testIndex]

In [218]:
testDataAll["PREDICT_SVM"] = predictValue

In [219]:
testDataAll

,name,gubun,score,visitCount,blogCount,name2,tenCen,twoCen,threeCen,fourCen,fiveCen,sixCen,womanRatio,manRatio,category,labelcode,sex,PREDICT_SVM
149,이와타,일본식라면,4.38,322,1,이와타,2.658875e+08,6.472144e+08,6.150637e+08,5.693206e+08,3.686238e+08,1.775701e+08,47,53,일식,6,0,50
21,바스버거 여의도점,햄버거,4.51,1239,787,바스버거 여의도점,1.389610e+08,3.339884e+08,2.976373e+08,3.139040e+08,2.482289e+08,1.549832e+08,50,50,양식,7,0,50
48,여의도 셋째집,돼지고기구이,4.63,435,602,여의도 셋째집,8.814341e+07,3.034087e+08,3.208359e+08,3.684221e+08,2.792682e+08,1.790617e+08,47,53,한식,9,0,50
39,채윤희,포장마차,4.32,416,536,채윤희,5.528355e+07,3.396267e+08,2.881440e+08,2.252818e+08,1.504081e+08,1.162729e+08,52,48,주류,5,1,50
140,승남집,소고기구이,4.61,498,678,승남집,1.175246e+08,6.311617e+08,6.056372e+08,6.095004e+08,4.667012e+08,1.775701e+08,47,53,한식,9,0,50
123,갓포아키 여의도점,일식당,4.47,188,308,갓포아키 여의도점,5.528355e+07,3.060620e+08,3.293934e+08,3.214377e+08,2.599435e+08,1.755813e+08,46,54,일식,6,0,50
1,콘래드서울 버티고,바(BAR),4.29,978,767,콘래드서울 버티고,1.236358e+08,5.194564e+08,5.262806e+08,5.397529e+08,4.666157e+08,2.942692e+08,56,44,주류,5,1,50
98,3일한우국밥,국밥,4.34,3060,503,3일한우국밥,1.175246e+08,5.282787e+08,5.808341e+08,6.382580e+08,5.945352e+08,6.501908e+08,44,56,한식,9,0,44
42,영일분식,종합분식,4.36,1809,1293,영일분식,1.386320e+08,6.190227e+08,6.600569e+08,8.072396e+08,7.683725e+08,5.530244e+08,50,50,분식,0,0,50
72,느루집,닭갈비,4.28,75,32,느루집,1.175246e+08,6.202831e+08,6.183396e+08,6.300763e+08,4.949187e+08,3.294991e+08,50,50,한식,9,0,50


In [222]:
from sklearn.preprocessing import MaxAbsScaler

# 절대값
scaler = MaxAbsScaler()

scaler = scaler.fit_transform(data)

# 교차검증시
scaler.fit(transform)
X_train = scaler.transform(trainingData_features)
X_test = scaler.transform(testData_features)

NameError: name 'data' is not defined